In [1]:
from __future__ import print_function, division
import os
import torch
import pandas as pd
from skimage import io, transform
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils
import torch.nn as nn
import torch.optim as optim
# Ignore warnings
import warnings
warnings.filterwarnings("ignore")

import torch.nn.functional as F
torch.set_printoptions(linewidth=120)

import pickle as serializer


In [2]:
with open('DataDict.txt', 'rb') as f:
    Data = serializer.load(f)
for batch_idx in enumerate(Data):
    print(batch_idx)

    if batch_idx == 3:
        break


(0, 'image')
(1, 'steer')
(2, 'throttle')
(3, 'direction')
(4, 'command')


In [3]:
with open('DataDict.txt', 'rb') as f:
    dataset = serializer.load(f)

class AISProject(Dataset):
    """ AIS Project dataset."""
    def __init__(self, dictionary, root_dir, transform = None):

        self.dictionary = pd.DataFrame.from_dict(dictionary)
        self.root_dir = root_dir
        self.transform = transform
        print(len(dictionary))
        
    def __len__(self):
        return len(self.dictionary)

    def __getitem__(self, idx):
        img_name = os.path.join(self.root_dir,self.dictionary.iloc[idx,0])
        image = io.imread(img_name)
        steer = torch.tensor(self.dictionary.iloc[idx,1],dtype=torch.float64, requires_grad=False)
        speed = torch.tensor(self.dictionary.iloc[idx,2],dtype = torch.float64)
        direction = torch.tensor(self.dictionary.iloc[idx,3],dtype = torch.long)
        command = torch.tensor(self.dictionary.iloc[idx,4],dtype = torch.long)
        sample = {'image':image, 'steer': steer, 'speed': speed,'direction':direction, 'command':command }
        
        if self.transform:
            sample = self.transform(sample)
            
        return sample

In [4]:
class ToTensor(object):
    """Convert ndarrays in sample to Tensors."""
    
    def __call__(self, sample):
        image, steer, speed, direction, command = sample['image'], sample['steer'], sample['speed'], sample['direction'], sample['command']
        image = image.transpose((2,0,1))
        return {'image': torch.from_numpy(image), 
                'steer': torch.tensor(steer),
               'speed': torch.tensor(speed),
               'direction':torch.tensor(direction),
               'command':torch.tensor(command)}

In [5]:
transformed_dataset = AISProject(dictionary = dataset, root_dir='Bilder/', 
                                transform = transforms.Compose([ToTensor()]))
transformed_dataset2 = AISProject(dictionary = dataset, root_dir='Bilder/', 
                                transform = transforms.Compose([ToTensor()]))
for i in range(len(transformed_dataset)):
    sample = transformed_dataset[i]
    
    print(i, sample['image'].size(), sample['steer'].size())
    print(sample['direction'])
    if i == 3:
        break

5
5
0 torch.Size([3, 128, 640]) torch.Size([])
tensor(2)
1 torch.Size([3, 128, 640]) torch.Size([])
tensor(2)
2 torch.Size([3, 128, 640]) torch.Size([])
tensor(2)
3 torch.Size([3, 128, 640]) torch.Size([])
tensor(1)


In [6]:
dataloader = DataLoader(transformed_dataset, batch_size=32, 
                        shuffle=False, pin_memory=True, num_workers=0)
dataloader2 = DataLoader(transformed_dataset2, batch_size=1, 
                        shuffle=False, pin_memory=True, num_workers=0)

for batch_idx, dataset in enumerate(dataloader):
    print(batch_idx)
    print(dataset['image'].size())
    print(dataset['command'].size())
    print(dataset['direction'])
    if batch_idx == 3:
        break
'''
def show_labels_batch(sample_batched):
    """Show image with labels for a batch of samples."""
    images_batch, labels_batch = \
        sample_batched['image'], sample_batched['labels']
    batch_size = len(images_batch)
    im_size = images_batch.size(2)
    
    grid = utils.make_grid(images_batch)
    plt.imshow(grid.numpy().transpose((1,2,0)))



for i_batch, sample_batched in enumerate(dataloader):
    print(i_batch, sample_batched['image'].size(), 
         sample_batched['labels'].size())

    if i_batch == 1:
        plt.figure(figsize=(40,40))
        show_labels_batch(sample_batched)
        plt.axis('off')
        plt.ioff()
        plt.show()
        break
'''
        

0
torch.Size([32, 3, 128, 640])
torch.Size([32])
tensor([2, 2, 2, 1, 1, 1, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 1, 1, 2, 2, 1, 1, 1, 1, 2, 1])
1
torch.Size([32, 3, 128, 640])
torch.Size([32])
tensor([1, 1, 1, 2, 2, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 2, 2, 1, 2, 2, 2, 2, 2, 2, 2, 2])
2
torch.Size([32, 3, 128, 640])
torch.Size([32])
tensor([2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 2, 3, 2, 2, 2, 2, 2, 2, 2, 1, 2, 2, 1, 1])
3
torch.Size([32, 3, 128, 640])
torch.Size([32])
tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 2, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 1, 2, 3, 2, 2, 2, 2])


'\ndef show_labels_batch(sample_batched):\n    """Show image with labels for a batch of samples."""\n    images_batch, labels_batch =         sample_batched[\'image\'], sample_batched[\'labels\']\n    batch_size = len(images_batch)\n    im_size = images_batch.size(2)\n    \n    grid = utils.make_grid(images_batch)\n    plt.imshow(grid.numpy().transpose((1,2,0)))\n\n\n\nfor i_batch, sample_batched in enumerate(dataloader):\n    print(i_batch, sample_batched[\'image\'].size(), \n         sample_batched[\'labels\'].size())\n\n    if i_batch == 1:\n        plt.figure(figsize=(40,40))\n        show_labels_batch(sample_batched)\n        plt.axis(\'off\')\n        plt.ioff()\n        plt.show()\n        break\n'

In [7]:
device = torch.device("cuda")
# Definition of the network architecture
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=5, padding=2)    # FYI: In the lecture I forgot to add the padding, thats why the feature size calculation was wrong
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.conv3 = nn.Conv2d(64, 64, kernel_size=3, padding=1)
        self.conv4 = nn.Conv2d(64, 128, kernel_size=3, padding=1)
        self.conv5 = nn.Conv2d(128, 128, kernel_size=3, padding=1)
        self.conv6 = nn.Conv2d(128, 128, kernel_size=3, padding=1)
        self.conv7 = nn.Conv2d(128, 256, kernel_size=3, padding=1)
        self.conv8 = nn.Conv2d(256, 256, kernel_size=3, padding=1)
        self.conv8_drop = nn.Dropout2d()
        self.fc1 = nn.Linear(327680, 50)
        self.fc2 = nn.Linear(50, 5)
    def forward(self, x):

        x = F.relu(self.conv1(x))
        x = F.max_pool2d(F.relu(self.conv2(x)),2)
        x = F.relu(self.conv3(x))
        x = F.max_pool2d(F.relu(self.conv4(x)),2)
        x = F.relu(self.conv5(x))
        x = F.relu(self.conv6(x))
        x = F.relu(self.conv7(x))
        x = F.relu(self.conv8_drop(self.conv8(x)))
        x = F.max_pool2d(x,2)
        x = x.view(-1, 327680)   # Flatten data for fully connected layer. Input size is 28*28, we have 2 pooling layers so we pool the spatial size down to 7*7. With 20 feature maps as the output of the previous conv we have in total 7x7x20 = 980 features.
        x = F.relu(self.fc1(x))
        x = F.dropout(x, training=self.training)
        x = self.fc2(x)
        return F.log_softmax(x, dim=1)
        '''
        x = F.relu(self.conv1(x))
        x = F.relu(F.max_pool2d(self.conv2(x),2))
        x = F.relu(self.conv3(x))
        x = F.relu(F.max_pool2d(self.conv4(x),2))
        x = F.relu(self.conv5(x))
        x = F.relu(self.conv6(x))
        x = F.relu(self.conv7(x))
        x = F.relu(F.max_pool2d(self.conv8_drop(self.conv8(x)), 2))
        x = x.view(-1, 327680)
        x = F.relu(self.fc1(x))
        x = F.dropout(x, training=self.training)
        x = self.fc2(x)
        return F.softmax(x, dim=1)
        '''
# We create the network, shift it on the GPU and define a optimizer on its parameters
model = Net().to(device)
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.5)
#optimizer = optim.Adam(model.parameters(), lr=0.01)

In [8]:
# This function trains the neural network for one epoch
def train(epoch):
    model.train()
    for batch_idx, sample in enumerate(dataloader):
        # Move the input and target data on the GPU
        sample['image'] = sample['image'].type('torch.FloatTensor')
        #sample['direction'] = sample['direction'].type('torch.FloatTensor')
        
        data, target = sample['image'].to(device), sample['steer'].to(device)  # .cuda() works too instead of .to(device)
        #loss = nn.L1Loss()
        # Zero out gradients from previous step
        optimizer.zero_grad()
        output = model(data)
        #print(output.type())
        #print("target")
        #print(target.type())
        #output = output.squeeze()
        loss = F.nll_loss(output, target)
        #loss = F.smooth_l1_loss(output, target)
        #loss = loss(output, target)
        loss.backward()
        # Adjusting the parameters according to the loss function
        optimizer.step()
        if batch_idx % 10 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(dataloader.dataset),
                100. * batch_idx / len(dataloader), loss.item()))


In [ ]:
def test():
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for sample in dataloader2:
            # Move the input and target data on the GPU
            sample['image'] = sample['image'].type('torch.FloatTensor')
            #sample['direction'] = sample['direction'].type('torch.FloatTensor')
            #loss = nn.L1Loss()
            data, target = sample['image'].to(device), sample['steer'].to(device)  # .cuda() works too instead of .to(device)
            output = model(data)
            #output = output.squeeze()
            #print(output)
            test_loss += F.nll_loss(output, target).item()
            #test_loss += F.smooth_l1_loss(output, target, size_average=False).item() # sum up batch loss
            #test_loss += loss(output, target).item()
            pred = output.max(1, keepdim=True)[1]
            #print(pred)
            print(target)
            #pred = output # get the index of the max log-probability
            '''
            if not pred.eq(target.view_as(pred)):   ## If you just want so see the failing examples
                cv_mat = data.cuda().data.squeeze().numpy()
                cv_mat = cv2.resize(cv_mat, (400, 400))
                cv2.imshow("test image", cv_mat)
                print("Target label is : %d" % target.cuda().item())
                print("Predicted label is : %d" % (pred.cuda().data.item()))
                cv2.waitKey()
            '''
            correct += pred.eq(target.view_as(pred)).sum().item()


    test_loss /= len(dataloader2.dataset)
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(dataloader2.dataset),
        100. * correct / len(dataloader2.dataset)))
'''
num_train_epochs = 2
for epoch in range(1, num_train_epochs + 1):
    train(epoch)
'''
test()

tensor([0], device='cuda:0')
tensor([0], device='cuda:0')
tensor([0], device='cuda:0')
tensor([0], device='cuda:0')
tensor([0], device='cuda:0')
tensor([0], device='cuda:0')
tensor([0], device='cuda:0')
tensor([0], device='cuda:0')
tensor([0], device='cuda:0')
tensor([0], device='cuda:0')
tensor([0], device='cuda:0')
tensor([0], device='cuda:0')
tensor([0], device='cuda:0')
tensor([0], device='cuda:0')
tensor([0], device='cuda:0')
tensor([0], device='cuda:0')
tensor([0], device='cuda:0')
tensor([0], device='cuda:0')
tensor([0], device='cuda:0')
tensor([0], device='cuda:0')
tensor([0], device='cuda:0')
tensor([0], device='cuda:0')
tensor([0], device='cuda:0')
tensor([0], device='cuda:0')
tensor([0], device='cuda:0')
tensor([0], device='cuda:0')
tensor([0], device='cuda:0')
tensor([0], device='cuda:0')
tensor([0], device='cuda:0')
tensor([0], device='cuda:0')
tensor([0], device='cuda:0')
tensor([0], device='cuda:0')
tensor([0], device='cuda:0')
tensor([0], device='cuda:0')
tensor([0], de

tensor([0], device='cuda:0')
tensor([0], device='cuda:0')
tensor([0], device='cuda:0')
tensor([0], device='cuda:0')
tensor([0], device='cuda:0')
tensor([0], device='cuda:0')
tensor([0], device='cuda:0')
tensor([0], device='cuda:0')
tensor([0], device='cuda:0')
tensor([0], device='cuda:0')
tensor([0], device='cuda:0')
tensor([0], device='cuda:0')
tensor([0], device='cuda:0')
tensor([0], device='cuda:0')
tensor([0], device='cuda:0')
tensor([0], device='cuda:0')
tensor([0], device='cuda:0')
tensor([0], device='cuda:0')
tensor([0], device='cuda:0')
tensor([0], device='cuda:0')
tensor([0], device='cuda:0')
tensor([0], device='cuda:0')
tensor([0], device='cuda:0')
tensor([0], device='cuda:0')
tensor([0], device='cuda:0')
tensor([0], device='cuda:0')
tensor([0], device='cuda:0')
tensor([0], device='cuda:0')
tensor([0], device='cuda:0')
tensor([0], device='cuda:0')
tensor([0], device='cuda:0')
tensor([0], device='cuda:0')
tensor([0], device='cuda:0')
tensor([0], device='cuda:0')
tensor([0], de

tensor([0], device='cuda:0')
tensor([0], device='cuda:0')
tensor([0], device='cuda:0')
tensor([0], device='cuda:0')
tensor([0], device='cuda:0')
tensor([0], device='cuda:0')
tensor([0], device='cuda:0')
tensor([0], device='cuda:0')
tensor([0], device='cuda:0')
tensor([0], device='cuda:0')
tensor([0], device='cuda:0')
tensor([0], device='cuda:0')
tensor([0], device='cuda:0')
tensor([0], device='cuda:0')
tensor([0], device='cuda:0')
tensor([0], device='cuda:0')
tensor([0], device='cuda:0')
tensor([0], device='cuda:0')
tensor([0], device='cuda:0')
tensor([0], device='cuda:0')
tensor([0], device='cuda:0')
tensor([0], device='cuda:0')
tensor([0], device='cuda:0')
tensor([0], device='cuda:0')
tensor([0], device='cuda:0')
tensor([0], device='cuda:0')
tensor([0], device='cuda:0')
tensor([0], device='cuda:0')
tensor([0], device='cuda:0')
tensor([0], device='cuda:0')
tensor([0], device='cuda:0')
tensor([0], device='cuda:0')
tensor([0], device='cuda:0')
tensor([0], device='cuda:0')
tensor([0], de

tensor([0], device='cuda:0')
tensor([0], device='cuda:0')
tensor([0], device='cuda:0')
tensor([0], device='cuda:0')
tensor([0], device='cuda:0')
tensor([0], device='cuda:0')
tensor([0], device='cuda:0')
tensor([0], device='cuda:0')
tensor([0], device='cuda:0')
tensor([0], device='cuda:0')
tensor([0], device='cuda:0')
tensor([0], device='cuda:0')
tensor([0], device='cuda:0')
tensor([0], device='cuda:0')
tensor([0], device='cuda:0')
tensor([0], device='cuda:0')
tensor([0], device='cuda:0')
tensor([0], device='cuda:0')
tensor([0], device='cuda:0')
tensor([0], device='cuda:0')
tensor([0], device='cuda:0')
tensor([0], device='cuda:0')
tensor([0], device='cuda:0')
tensor([0], device='cuda:0')
tensor([0], device='cuda:0')
tensor([0], device='cuda:0')
tensor([0], device='cuda:0')
tensor([0], device='cuda:0')
tensor([0], device='cuda:0')
tensor([0], device='cuda:0')
tensor([0], device='cuda:0')
tensor([0], device='cuda:0')
tensor([0], device='cuda:0')
tensor([0], device='cuda:0')
tensor([0], de

tensor([0], device='cuda:0')
tensor([0], device='cuda:0')
tensor([0], device='cuda:0')
tensor([0], device='cuda:0')
tensor([0], device='cuda:0')
tensor([0], device='cuda:0')
tensor([0], device='cuda:0')
tensor([0], device='cuda:0')
tensor([0], device='cuda:0')
tensor([0], device='cuda:0')
tensor([0], device='cuda:0')
tensor([0], device='cuda:0')
tensor([0], device='cuda:0')
tensor([0], device='cuda:0')
tensor([0], device='cuda:0')
tensor([0], device='cuda:0')
tensor([0], device='cuda:0')
tensor([0], device='cuda:0')
tensor([0], device='cuda:0')
tensor([0], device='cuda:0')
tensor([0], device='cuda:0')
tensor([0], device='cuda:0')
tensor([0], device='cuda:0')
tensor([0], device='cuda:0')
tensor([0], device='cuda:0')
tensor([0], device='cuda:0')
tensor([0], device='cuda:0')
tensor([0], device='cuda:0')
tensor([0], device='cuda:0')
tensor([0], device='cuda:0')
tensor([0], device='cuda:0')
tensor([0], device='cuda:0')
tensor([0], device='cuda:0')
tensor([0], device='cuda:0')
tensor([0], de

...

tensor([0], device='cuda:0')
tensor([0], device='cuda:0')
tensor([0], device='cuda:0')
tensor([0], device='cuda:0')
tensor([0], device='cuda:0')
tensor([0], device='cuda:0')
tensor([0], device='cuda:0')
tensor([0], device='cuda:0')
tensor([0], device='cuda:0')
tensor([0], device='cuda:0')
tensor([0], device='cuda:0')
tensor([0], device='cuda:0')
tensor([0], device='cuda:0')
tensor([0], device='cuda:0')
tensor([0], device='cuda:0')
tensor([0], device='cuda:0')
tensor([0], device='cuda:0')
tensor([0], device='cuda:0')
tensor([0], device='cuda:0')
tensor([0], device='cuda:0')
tensor([0], device='cuda:0')
tensor([0], device='cuda:0')
tensor([0], device='cuda:0')
tensor([0], device='cuda:0')
tensor([0], device='cuda:0')
tensor([0], device='cuda:0')
tensor([0], device='cuda:0')
tensor([0], device='cuda:0')
tensor([0], device='cuda:0')
tensor([0], device='cuda:0')
tensor([0], device='cuda:0')
tensor([0], device='cuda:0')
tensor([0], device='cuda:0')
tensor([0], device='cuda:0')
tensor([0], de

tensor([0], device='cuda:0')
tensor([0], device='cuda:0')
tensor([0], device='cuda:0')
tensor([0], device='cuda:0')
tensor([0], device='cuda:0')
tensor([0], device='cuda:0')
tensor([0], device='cuda:0')
tensor([0], device='cuda:0')
tensor([0], device='cuda:0')
tensor([0], device='cuda:0')
tensor([0], device='cuda:0')
tensor([0], device='cuda:0')
tensor([0], device='cuda:0')
tensor([0], device='cuda:0')
tensor([0], device='cuda:0')
tensor([0], device='cuda:0')
tensor([0], device='cuda:0')
tensor([0], device='cuda:0')
tensor([0], device='cuda:0')
tensor([0], device='cuda:0')
tensor([0], device='cuda:0')
tensor([0], device='cuda:0')
tensor([0], device='cuda:0')
tensor([0], device='cuda:0')
tensor([0], device='cuda:0')
tensor([0], device='cuda:0')
tensor([0], device='cuda:0')
tensor([0], device='cuda:0')
tensor([0], device='cuda:0')
tensor([0], device='cuda:0')
tensor([0], device='cuda:0')
tensor([0], device='cuda:0')
tensor([0], device='cuda:0')
tensor([0], device='cuda:0')
tensor([0], de

tensor([0], device='cuda:0')
tensor([0], device='cuda:0')
tensor([0], device='cuda:0')
tensor([0], device='cuda:0')
tensor([0], device='cuda:0')
tensor([0], device='cuda:0')
tensor([0], device='cuda:0')
tensor([0], device='cuda:0')
tensor([0], device='cuda:0')
tensor([0], device='cuda:0')
tensor([0], device='cuda:0')
tensor([0], device='cuda:0')
tensor([0], device='cuda:0')
tensor([0], device='cuda:0')
tensor([0], device='cuda:0')
tensor([0], device='cuda:0')
tensor([0], device='cuda:0')
tensor([0], device='cuda:0')
tensor([0], device='cuda:0')
tensor([0], device='cuda:0')
tensor([0], device='cuda:0')
tensor([0], device='cuda:0')
tensor([0], device='cuda:0')
tensor([0], device='cuda:0')
tensor([0], device='cuda:0')
tensor([0], device='cuda:0')
tensor([0], device='cuda:0')
tensor([0], device='cuda:0')
tensor([0], device='cuda:0')
tensor([0], device='cuda:0')
tensor([0], device='cuda:0')
tensor([0], device='cuda:0')
tensor([0], device='cuda:0')
tensor([0], device='cuda:0')
tensor([0], de

tensor([0], device='cuda:0')
tensor([0], device='cuda:0')
tensor([0], device='cuda:0')
tensor([0], device='cuda:0')
tensor([0], device='cuda:0')
tensor([0], device='cuda:0')
tensor([0], device='cuda:0')
tensor([0], device='cuda:0')
tensor([0], device='cuda:0')
tensor([0], device='cuda:0')
tensor([0], device='cuda:0')
tensor([0], device='cuda:0')
tensor([0], device='cuda:0')
tensor([0], device='cuda:0')
tensor([0], device='cuda:0')
tensor([0], device='cuda:0')
tensor([0], device='cuda:0')
tensor([0], device='cuda:0')
tensor([0], device='cuda:0')
tensor([0], device='cuda:0')
tensor([0], device='cuda:0')
tensor([0], device='cuda:0')
tensor([0], device='cuda:0')
tensor([0], device='cuda:0')
tensor([0], device='cuda:0')
tensor([0], device='cuda:0')
tensor([0], device='cuda:0')
tensor([0], device='cuda:0')
tensor([0], device='cuda:0')
tensor([0], device='cuda:0')
tensor([0], device='cuda:0')
tensor([0], device='cuda:0')
tensor([0], device='cuda:0')
tensor([0], device='cuda:0')
tensor([0], de

tensor([0], device='cuda:0')
tensor([0], device='cuda:0')
tensor([0], device='cuda:0')
tensor([0], device='cuda:0')
tensor([0], device='cuda:0')
tensor([0], device='cuda:0')
tensor([0], device='cuda:0')
tensor([0], device='cuda:0')
tensor([0], device='cuda:0')
tensor([0], device='cuda:0')
tensor([0], device='cuda:0')
tensor([0], device='cuda:0')
tensor([0], device='cuda:0')
tensor([0], device='cuda:0')
tensor([0], device='cuda:0')
tensor([0], device='cuda:0')
tensor([0], device='cuda:0')
tensor([0], device='cuda:0')
tensor([0], device='cuda:0')
tensor([0], device='cuda:0')
tensor([0], device='cuda:0')
tensor([0], device='cuda:0')
tensor([0], device='cuda:0')
tensor([0], device='cuda:0')
tensor([0], device='cuda:0')
tensor([0], device='cuda:0')
tensor([0], device='cuda:0')
tensor([0], device='cuda:0')
tensor([0], device='cuda:0')
tensor([0], device='cuda:0')
tensor([0], device='cuda:0')
tensor([0], device='cuda:0')
tensor([0], device='cuda:0')
tensor([0], device='cuda:0')
tensor([0], de

In [10]:
loss = nn.L1Loss()
input1 = torch.tensor([-1.88],requires_grad=True,dtype=torch.float32)
input2 = torch.tensor([-1.88],requires_grad=True,dtype=torch.float32)
target = torch.tensor([1],dtype=torch.float64)
b = input2.max(0, keepdim=True)[0]
#output = loss(input1,input2)
print(b)
#output.backward()
#print(output)
print(input1.eq(input2).sum().item())

#x = output.max(0,keepdim=True)[1]
m = input2.view_as(0)
print("m")
print(m)

correct = output.eq(target.view_as(output)).sum().item()
print(correct)
#print(x)

tensor([-1.8800], grad_fn=<MaxBackward0>)
1


TypeError: view_as(): argument 'other' (position 1) must be Tensor, not int

In [ ]:
counter = 0
all = 0
a = torch.tensor([0],dtype=torch.float64)
for batch_idx, sample in enumerate(dataloader2):
    all += 1
    b = sample['steer']
    if b==a:
        counter += 1
print(counter*100/all)      